In [2]:
import torch
from torch.nn import functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
MODEL = "/scratch/bchk/aguha/models/llama3p1_8b_base"
DEVICE = "cuda"
n_samp = 20

# load in tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token 

# load in model
model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.bfloat16,
).to(device=DEVICE)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [145]:
import math
import re
from typing import Dict, List, Tuple

def calculate_path_distance(coords: Dict[str, List[float]], path: List[int]) -> float:
    """
    Calculate the total distance of a path through the given coordinates.
    
    Args:
        coords: Dictionary mapping node IDs to coordinate pairs.
        path: List of node IDs representing the order of visitation.
        
    Returns:
        Total Euclidean distance of the path.
    """
    total_distance = 0
    for i in range(len(path)):
        current_node = path[i]
        next_node = path[0] if i == len(path) - 1 else path[i + 1]
        
        # Fix the coordinate lookup to use the actual node IDs from the path
        x1, y1 = coords[str(current_node+1)]
        x2, y2 = coords[str(next_node+1)]
        
        # Calculate Euclidean distance
        distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        total_distance += distance
    
    return total_distance

def generate_solutions(prompt_dict, tokenizer, model, max_new_tokens=300, temperature=0.2, n_samples=20):

    # Convert the conversation into a formatted string for tokenization
    formatted_prompt = ""
    for message in prompt_dict:
        role = message["role"]
        content = message["content"]
        if role == "system":
            formatted_prompt += f"[System]: {content}\n"
        elif role == "user":
            formatted_prompt += f"[User]: {content}\n"
        elif role == "assisstant":
            formatted_prompt += f"[Assistant]: {content}\n"

    # Tokenize and move to CUDA
    inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True)["input_ids"].to("cuda")

    # Generate n_samples number of solutions
    outputs = model.generate(
        inputs,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        num_return_sequences=n_samples
    )

    # Decode only the new tokens generated after the input prompt
    solutions = [tokenizer.decode(output[inputs.shape[1]:], skip_special_tokens=True) for output in outputs]

    return solutions

       
def extract_trace(solution_text: str) -> List[int]:
    """
    Extract the first valid trace from the solution text.
    
    Args:
        solution_text: The generated solution text.
        
    Returns:
        List of integers representing the trace.
    """
    # Split the text into lines for line-by-line processing
    lines = solution_text.strip().split('\n')
    
    for line in lines:
        # Skip empty lines
        if not line.strip():
            continue
        
        # Look for patterns like "0,1,2,3,4" or "Trace 0,4,1,2,3"
        trace_patterns = [
            r'(\d+(?:,\s*\d+)+)',  # Basic comma-separated numbers
            r'[Tt]race\s*(\d+(?:,\s*\d+)+)',  # Prefixed with "Trace"
            r'trace\s*is\s*:\s*(\d+(?:,\s*\d+)+)',  # "trace is: 0,1,2,3"
        ]
        
        for pattern in trace_patterns:
            match = re.search(pattern, line)
            if match:
                trace_str = match.group(1)
                # Clean up the trace string and convert to integers
                try:
                    trace = [int(num.strip()) for num in trace_str.split(',')]
                    # Ensure we have a valid trace (at least 2 nodes)
                    if len(trace) >= 2:
                        return trace
                except ValueError:
                    # If conversion fails, continue to the next pattern
                    continue
    
    # If no valid trace is found, return an empty list
    return []


def test_model(conversation, tokenizer, model, coords,solution, max_iterations=20, n_samples=5):
    """
    Iteratively tests the model by generating multiple traces and evaluating their distances.
    
    Args:
        conversation: List of message dictionaries with role and content.
        tokenizer: The tokenizer for the model.
        model: The language model.
        coords: Dictionary of node coordinates.
        max_iterations: Maximum number of iterations before stopping.
    
    Returns:
        Updated conversation history with appended interactions.
    """
    tested_traces = set()
    best_distance = float("inf")
    best_trace = []
    iteration = 0

    while iteration < max_iterations:
        iteration += 1
        print(f"\nIteration {iteration}")
        
        # Generate 5 different solutions
        solutions = generate_solutions(
            conversation, 
            tokenizer, 
            model, 
            temperature=0.5, 
            n_samples=n_samples, 
            max_new_tokens=100
        )
        
        # Track the best solution from this batch
        batch_best_distance = float("inf")
        batch_best_trace = []
        
        # Evaluate each solution
        for i, solution_text in enumerate(solutions):
            trace = extract_trace(solution_text)
            
            if not trace:
                print(f"  Solution {i+1}: No valid trace extracted")
                continue
                
            if len(trace) != len(set(trace)):
                print(f"  Solution {i+1}: Invalid trace (duplicate nodes)")
                continue
                
            if tuple(trace) in tested_traces:
                print(f"  Solution {i+1}: Duplicate trace {trace}")
                continue
            
            # Calculate the distance
            try:
                distance = calculate_path_distance(coords, trace)
                print(f"  Solution {i+1}: Trace {trace} with distance {distance:.2f}")
                
                # Update batch best
                if distance < batch_best_distance:
                    batch_best_distance = distance
                    batch_best_trace = trace
                
                # Add to tested traces
                tested_traces.add(tuple(trace))
                
            except KeyError as e:
                print(f"  Solution {i+1}: Error calculating distance for trace {trace} - {e}")
        
        # Check if we found any valid traces in this batch
        if not batch_best_trace:
            print("No valid traces found in this batch. Stopping.")
            break
        
        # Check if the best trace from this batch improves on the overall best
        if batch_best_distance >= best_distance:
            print(f"No improvement over best distance {best_distance:.2f}. Stopping.")
            break
        
        # Update best distance and trace
        best_distance = batch_best_distance
        best_trace = batch_best_trace
        
        # Append new conversation turns
        trace_str = ", ".join(map(str, best_trace))
        conversation.append({"role": "assistant", "content": trace_str})
        conversation.append({
            "role": "user", 
            "content": f"Distance: {best_distance:.2f}\nPlease find a trace with a shorter distance:"
        })

    print(f"\nFinal best trace: {best_trace}")
    print(f"Final best distance: {best_distance:.2f}")
    
    solved = 0
    if best_distance-.1 <= solution['distance']:
        solved = 1
        print("Optimal Solution Found")
    
    return conversation, solved



In [146]:
with open("tsp_dataset_100_problems.json", "r") as f:
    problems = f.read()
problems = json.loads(problems)
problems['size_5'][0]['coordinates']

{'1': [63, -72],
 '2': [-94, 89],
 '3': [-30, -38],
 '4': [-43, -65],
 '5': [88, -74]}

In [148]:
with open("tsp_llm_prompts_with_matrix_multi.json", "r") as f:
    
    prompts = f.read()
    
prompts = json.loads(prompts)
prompt = prompts['size_5'][0]['prompt']
prompt

[{'role': 'system',
  'content': 'You are given a list of points with coordinates below: (1): (99, 35), (2): (-100, 53), (3): (-18, 25), (4): (-96, -72), (5): (-8, -22).\n\nDistance Matrix:\n       0    1    2    3    4\n 0:  0.0 199.8 117.4 222.4 121.2 \n 1: 199.8  0.0 86.6 125.1 118.7 \n 2: 117.4 86.6  0.0 124.5 48.1 \n 3: 222.4 125.1 124.5  0.0 101.2 \n 4: 121.2 118.7 48.1 101.2  0.0 \n\nHere is a sample trace and its associated distance: \n\n0,3,2,4,1 length: 713\n'},
 {'role': 'user',
  'content': '\nGive me a new trace that is different from the above trace, and has a length lower than the above trace. The trace should traverse all nodesThe path must start at 0 will return to 0 at the end of the trace, which is included in the distance.'},
 {'role': 'assisstant', 'content': '0,4,1,2,3'},
 {'role': 'user',
  'content': 'Distance 673 \n Please find a trace with a shorter distance:'},
 {'role': 'assisstant', 'content': '0,3,4,2,1'},
 {'role': 'user',
  'content': 'Distance 658 \n Pl

In [141]:
prompts_5 = prompts['size_5']
problems_5 = problems['size_5']

correct = 0


for prompt_dict, problem_dict in zip(prompts_5, problems_5):
    
    prompt = prompt_dict["prompt"]
    coords = problem_dict["coordinates"]
    solution = problem_dict["solution"]
    
    results = test_model(prompt, tokenizer, model, coords, solution)
    
    correct += results[1]
    
    



Iteration 1
  Solution 1: Trace [0, 4, 1, 3, 2] with distance 560.61
  Solution 2: Trace [0, 4, 3, 1, 2] with distance 559.85
  Solution 3: Trace [0, 3, 4, 1, 2] with distance 723.10
  Solution 4: Trace [0, 2, 1, 3, 4] with distance 559.85
  Solution 5: Duplicate trace [0, 4, 3, 1, 2]

Iteration 2
  Solution 1: Duplicate trace [0, 3, 4, 1, 2]
  Solution 2: Trace [0, 3, 1, 2, 4] with distance 559.12
  Solution 3: Trace [0, 4, 2, 1, 3] with distance 559.12
  Solution 4: Trace [0, 2, 1, 4, 3] with distance 723.10
  Solution 5: Trace [0, 3, 2, 1, 4] with distance 547.81

Iteration 3
  Solution 1: No valid trace extracted
  Solution 2: No valid trace extracted
  Solution 3: No valid trace extracted
  Solution 4: No valid trace extracted
  Solution 5: No valid trace extracted
No valid traces found in this batch. Stopping.

Final best trace: [0, 3, 2, 1, 4]
Final best distance: 547.81
Optimal Solution Found

Iteration 1
  Solution 1: Trace [0, 3, 1, 2, 4] with distance 804.03
  Solution 2: T

In [142]:
print(correct)

17


In [149]:
prompts_10 = prompts['size_10']
problems_10 = problems['size_10']

correct = 0


for prompt_dict, problem_dict in zip(prompts_10, problems_10):
    
    prompt = prompt_dict["prompt"]
    coords = problem_dict["coordinates"]
    solution = problem_dict["solution"]
    
    results = test_model(prompt, tokenizer, model, coords, solution, n_samples=20)
    
    correct += results[1]
    
    



Iteration 1
  Solution 1: Trace [0, 9, 1, 2, 4, 3, 5, 8, 6, 7] with distance 1226.39
  Solution 2: Trace [0, 5, 4, 1, 2, 3, 8, 9, 7, 6] with distance 1059.16
  Solution 3: Trace [0, 7, 5, 4, 3, 2, 9, 8, 1, 6] with distance 1042.80
  Solution 4: Trace [0, 7, 9, 5, 8, 1, 2, 4, 3, 6] with distance 1269.23
  Solution 5: Trace [0, 5, 4, 1, 3, 2, 9, 8, 7, 6] with distance 944.25
  Solution 6: Trace [0, 7, 8, 9, 5, 4, 3, 2, 6, 1] with distance 997.84
  Solution 7: Trace [0, 5, 4, 3, 1, 2, 9, 8, 7, 6] with distance 1120.93
  Solution 8: Trace [0, 7, 5, 4, 3, 1, 2, 8, 9, 6] with distance 1299.65
  Solution 9: Duplicate trace [0, 5, 4, 1, 3, 2, 9, 8, 7, 6]
  Solution 10: Trace [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] with distance 1116.09
  Solution 11: Trace [0, 9, 1, 2, 4, 3, 8, 5, 7, 6] with distance 1287.61
  Solution 12: Trace [0, 5, 4, 3, 2, 1, 9, 8, 7, 6] with distance 1101.99
  Solution 13: Trace [0, 8, 7, 6, 5, 4, 3, 2, 1, 9] with distance 1239.12
  Solution 14: Trace [0, 4, 1, 3, 2, 9, 8, 5, 7,

In [144]:
print(correct)

0
